In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

from pathlib import Path
import pandas as pd
import numpy as np

fdir = Path.cwd()
print(fdir)
sys.path.append(str(fdir/'../src'))
from config import cfg

from build_df import load_rna
from merge_meta_files import load_crossref, load_pdx_meta
import load_data
from load_data import load_crossref, load_pdx_meta, load_pdx_meta2, load_pdx_meta_jc, load_rna, PDX_SAMPLE_COLS

/vol/ml/apartin/projects/pdx-histo/nbs


### Judith

In [2]:
jj = load_pdx_meta_jc()
print(jj.shape)
jj[:2]

(1198, 8)


,Sample,patient_id,specimen_id,sample_id,csite_src,ctype_src,csite,ctype
0,114434~197-R~A35YC3,114434,197-R,A35YC3,Musculoskeletal,Non-Rhabdo. soft tissue sarcoma,musculoskeletal,sarcoma/mesothelioma
1,114434~197-R~A36YC9,114434,197-R,A36YC9,Musculoskeletal,Non-Rhabdo. soft tissue sarcoma,musculoskeletal,sarcoma/mesothelioma


### Yitan

In [6]:
yy = load_pdx_meta()
print(yy.shape)
yy[:2]

(97, 7)


,patient_id,specimen_id,csite_src,ctype_src,csite,ctype,stage_or_grade
0,114434,197-R,Musculoskeletal,Non-Rhabdo. soft tissue sarcoma,musculoskeletal,sarcoma/mesothelioma,grade 3
1,114551,080-T,Head and Neck,Salivary gland cancer,head & neck,head & neck,grade 2


### Cross-ref

In [7]:
cref = load_crossref(drop_bad_slides=False)
print(cref.shape)
cref[:2]

(593, 5)


,model,patient_id,specimen_id,sample_id,image_id
0,114434~197-R,114434,197-R,A35YC3,27166
1,114434~197-R,114434,197-R,A36YC9,25127


### Merge slides meta (cref) and PDX meta

In [9]:
print(cref.shape)
print(yy.shape)
display(cref[:2])
display(yy[:2])

(593, 5)
(97, 7)


,model,patient_id,specimen_id,sample_id,image_id
0,114434~197-R,114434,197-R,A35YC3,27166
1,114434~197-R,114434,197-R,A36YC9,25127


,patient_id,specimen_id,csite_src,ctype_src,csite,ctype,stage_or_grade
0,114434,197-R,Musculoskeletal,Non-Rhabdo. soft tissue sarcoma,musculoskeletal,sarcoma/mesothelioma,grade 3
1,114551,080-T,Head and Neck,Salivary gland cancer,head & neck,head & neck,grade 2


In [10]:
# A few samples are missing
mrg_cols = ['patient_id', 'specimen_id']
m = cref.merge(yy[mrg_cols], on=mrg_cols, how='inner')
print(m.shape)
display(m[:2])

(584, 5)


,model,patient_id,specimen_id,sample_id,image_id
0,114434~197-R,114434,197-R,A35YC3,27166
1,114434~197-R,114434,197-R,A36YC9,25127


In [11]:
# Explore (merge and identify from which df the items are coming from)
# https://kanoki.org/2019/07/04/pandas-difference-between-two-dataframes/
mrg_outer = cref.merge(yy, on=mrg_cols, how='outer', indicator=True)
print('Outer merge', mrg_outer.shape)
print(mrg_outer['_merge'].value_counts())

miss = mrg_outer.loc[lambda x: x['_merge']=='left_only']
miss = miss.sort_values(mrg_cols, ascending=True)
print('\nMissing items', miss.shape)
display(miss)

Outer merge (593, 11)
both          584
left_only       9
right_only      0
Name: _merge, dtype: int64

Missing items (9, 11)


,model,patient_id,specimen_id,sample_id,image_id,csite_src,ctype_src,csite,ctype,stage_or_grade,_merge
45,146476~266-R,146476,266-R,E19YJ0,20252,NaN,NaN,NaN,NaN,NaN,left_only
46,146476~266-R,146476,266-R,E19YJ1,20307,NaN,NaN,NaN,NaN,NaN,left_only
47,146476~266-R,146476,266-R,E20F07,21545,NaN,NaN,NaN,NaN,NaN,left_only
48,146476~266-R,146476,266-R,E20F08F67,20976,NaN,NaN,NaN,NaN,NaN,left_only
49,146476~266-R,146476,266-R,ORIGINATOR,13946,NaN,NaN,NaN,NaN,NaN,left_only
148,237351~077-R,237351,077-R,AL-IR0,20729,NaN,NaN,NaN,NaN,NaN,left_only
149,237351~077-R,237351,077-R,AL-IR0_AL-M977,20467,NaN,NaN,NaN,NaN,NaN,left_only
150,237351~077-R,237351,077-R,AL-IR0_AL-M977_AL-M302,23429,NaN,NaN,NaN,NaN,NaN,left_only
151,237351~077-R,237351,077-R,AL-IR0_AL-M977_AL-M303_AL-ER0,26305,NaN,NaN,NaN,NaN,NaN,left_only


In [12]:
print(miss.patient_id.unique())
miss_found = jj[ jj.patient_id.isin(miss.patient_id.unique()) ]
miss_found = miss_found.drop(columns=['Sample', 'sample_id'])
miss_found = miss_found.drop_duplicates()
miss_found

['146476' '237351']


,patient_id,specimen_id,csite_src,ctype_src,csite,ctype
78,146476,266-R,Genitourinary,"Urothelial/bladder cancer, NOS",bladder/urothelial,bladder/urothelial
210,237351,077-R,Musculoskeletal,Ewing sarcoma/Peripheral PNET,musculoskeletal,sarcoma/mesothelioma


In [13]:
# Add the missing samples
yy_new = pd.concat([yy, miss_found], axis=0)
yy_new = yy_new.reset_index(drop=True)
print(yy_new.shape)

(99, 7)


In [14]:
# Now mege with the corrected file
mrg_cols = ['patient_id', 'specimen_id']
m = cref.merge(yy_new[mrg_cols], on=mrg_cols, how='inner')
print(m.shape)
display(m[:2])

(593, 5)


,model,patient_id,specimen_id,sample_id,image_id
0,114434~197-R,114434,197-R,A35YC3,27166
1,114434~197-R,114434,197-R,A36YC9,25127


In [15]:
# Save into file
yy_new.to_csv('../data/meta/PDX_Meta_Information2.csv', index=False)

# Merge for drug response
There drug response pair for which we don't have histology slides.

In [15]:
rsp = load_data.load_rsp()
rna = load_data.load_rna()
dd = load_data.load_dd()
cref = load_data.load_crossref()
pdx = load_data.load_pdx_meta2()


Dropped 77 duplicates.

Drop bad slides from image_id.
cref: (593, 5)
Bad slides: [45983, 83742, 83743, 22232, 21836, 20729]
cref: (587, 5)


In [16]:
print(rsp.shape)
print(rna.shape)
rsp_rna = rsp.merge(rna, on='Sample', how='inner')
print(rsp_rna.shape)

(2591, 4)
(1727, 981)
(2591, 984)


In [17]:
print(rsp_rna.shape)
print(dd.shape)
rsp_rna_dd = rsp_rna.merge(dd, left_on='Drug1', right_on='ID', how='inner').reset_index(drop=True)
print(rsp_rna_dd.shape)

(2591, 984)
(2142, 1617)
(2591, 2601)


In [18]:
print(pdx.shape)
print(rsp_rna_dd.shape)
rsp_rna_dd_pdx = pdx.merge(rsp_rna_dd, on=['patient_id', 'specimen_id'], how='inner')
print(rsp_rna_dd_pdx.shape)

(99, 7)
(2591, 2601)
(2591, 2606)


In [19]:
print(cref.shape)
print(rsp_rna_dd_pdx.shape)
df = cref.merge(rsp_rna_dd_pdx, on=PDX_SAMPLE_COLS, how='inner')
print(df.shape)

(587, 5)
(2591, 2606)
(2574, 2607)


In [26]:
# Explore (merge and identify from which df the items are coming from)
# https://kanoki.org/2019/07/04/pandas-difference-between-two-dataframes/
mrg_outer = cref.merge(rsp_rna_dd_pdx, on=PDX_SAMPLE_COLS, how='outer', indicator=True)
print('Outer merge', mrg_outer.shape)
print(mrg_outer['_merge'].value_counts())

miss_r = mrg_outer.loc[lambda x: x['_merge']=='right_only']
miss_r = miss_r.sort_values(PDX_SAMPLE_COLS, ascending=True)
print('\nMissing right items', miss_r.shape)

miss_l = mrg_outer.loc[lambda x: x['_merge']=='left_only']
miss_l = miss_l.sort_values(PDX_SAMPLE_COLS, ascending=True)
print('Missing left items ', miss_l.shape)

Outer merge (2772, 2608)
both          2574
left_only      181
right_only      17
Name: _merge, dtype: int64

Missing right items (17, 2608)
Missing left items  (181, 2608)


In [27]:
print(miss_r.patient_id.unique())
miss_found = jj[ jj.patient_id.isin(miss_r.patient_id.unique()) ]
print(miss_r.Response.value_counts())
display(miss_found)

['269878' '773659' 'BL0382']
0.0    16
1.0     1
Name: Response, dtype: int64


,Sample,patient_id,specimen_id,sample_id,csite_src,ctype_src,csite,ctype
264,269878~174-B~PN3,269878,174-B,PN3,Skin,Merkel cell tumor,skin,skin other
265,269878~174-B~PN3M26EN0FV4,269878,174-B,PN3M26EN0FV4,Skin,Merkel cell tumor,skin,skin other
266,269878~174-B~PN3M27,269878,174-B,PN3M27,Skin,Merkel cell tumor,skin,skin other
267,269878~174-B~PN3M29,269878,174-B,PN3M29,Skin,Merkel cell tumor,skin,skin other
812,773659~155-R~NQ6,773659,155-R,NQ6,Skin,Melanoma,skin,melanoma
813,773659~155-R~NQ6E39,773659,155-R,NQ6E39,Skin,Melanoma,skin,melanoma
814,773659~155-R~NQ8,773659,155-R,NQ8,Skin,Melanoma,skin,melanoma
815,773659~155-R~NQ8K16,773659,155-R,NQ8K16,Skin,Melanoma,skin,melanoma
816,773659~155-R~NQ8K17,773659,155-R,NQ8K17,Skin,Melanoma,skin,melanoma
817,773659~155-R~NQ8K19M745,773659,155-R,NQ8K19M745,Skin,Melanoma,skin,melanoma


In [29]:
miss_r[miss_r.Response==1]

,model,patient_id,specimen_id,sample_id,image_id,csite_src,ctype_src,csite,ctype,stage_or_grade,...,dd_TSRW10,dd_MW,dd_AMW,dd_WPath,dd_WPol,dd_Zagreb1,dd_Zagreb2,dd_mZagreb1,dd_mZagreb2,_merge
2755,269878~174-B,269878,174-B,PN3,NaN,Skin,Merkel cell tumor,skin,skin other,nan,...,102.40372,853.33093,7.551601,14767.0,129.0,346.0,429.0,23.36111,13.215278,right_only
